In [ ]:
import os
import requests

from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown
from openai import OpenAI

In [52]:
# Standard headers to fetch a website
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

def fetch_website_contents(url):
    """
    Return the title and contents of the website at the given url;
    truncate to 2,000 characters as a sensible limit
    """
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    title = soup.title.string if soup.title else "No title found"
    if soup.body:
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        text = soup.body.get_text(separator="\n", strip=True)
    else:
        text = ""
    return (title + "\n\n" + text)

In [30]:
# Load environment variables in a file called .env
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key
if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [64]:
website = fetch_website_contents("https://www.allrecipes.com/easy-5-ingredient-dinners-11799627")
print(website)

10 Easy Dinners You Won’t Believe Use Just 5 Ingredients

​
Skip to content
Allrecipes
Search
Please fill out this field.
Search the site
Please fill out this field.
Log In
My Account
Add a Recipe
Saved Recipes & Collections
Account Settings
Help
Log Out
Newsletters
Sweepstakes
Allrecipes
Search
Please fill out this field.
Dinners
Dinners
5-Ingredient Dinners
One-Pot Meals
Quick & Easy
30-Minute Meals
Family Dinners
Soups, Stews & Chili
Comfort Food
Main Dishes
Sheet Pan Dinners
View All
Meals
Meals
Breakfast & Brunch
Lunch
Healthy
Appetizers & Snacks
Salads
Side Dishes
Soups
Bread
Drinks
Desserts
View All
Ingredients
Ingredients
Chicken
Beef
Pork
Seafood
Pasta
Fruits
Vegetables
View All
Occasions
Occasions
Thanksgiving Appetizers
Thanksgiving Side Dishes
Thanksgiving Turkey
Thanksgiving Desserts
More Thanksgiving Recipes
View All
Cuisines
Cuisines
Mexican
Italian
Chinese
Indian
German
Greek
Filipino
Japanese
View All
Kitchen Tips
Kitchen Tips
Instant Pot
Air Fryer
Slow Cooker
Our Favo

In [ ]:
# Define System Prompt
system_prompt = """
You are a well-spoken cooking assistant that analyzes the contents of a food recipe website's meal list,
and finds a random recipe from within that list of recipes. You return the recipe name as a header, and if you cannot follow the link
use the internet to generate an appropriate recipe/dish summary or background in a well detailed paragraph.
Then you can return the list of ingredients followed by the cooking instructions for both conventional methods and air fryers 
(similarly, if you cannot follow the link, then use the internet to find or generate the appropriate ingredient list and instructions).
You do not need to select meals that include 'air fryer' in the name or already have air-fryer instructions. If a recipe does not have
air-fryer-related instructions, use the internet to find the best way to cook the recipe on an air fryer and generate the instructions.
Finally, display the information for that recipe. Format the ingredients (use this as a subheading) and instructions (use this as another subheading) 
into their own neatly organized tables.

You ignore text that might be website-navigation related that is not related to the link to the recipe.

You respond in markdown. Do not wrap the markdown in a code block!
"""

In [ ]:
# Define User Prompt

user_prompt_prefix = """
Here are the contents of a food recipe website's easy dinners list. Randomly choose one of the recipes. 
Return the recipe name, then provide a summary or background of the dish. Then return the ingredients, 
and cooking instructions for both conventional methods and air-frying.
"""

In [94]:
openai = OpenAI()

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt_prefix + website}
]

response = openai.chat.completions.create(model="gpt-5-nano", messages=messages)
Markdown(response.choices[0].message.content)

# Spaghetti alla Carbonara

Spaghetti alla Carbonara is a classic Roman pasta dish that blends crisp guanciale or pancetta with a creamy, cheese-llecked egg sauce. Traditional carbonara relies on the heat of the freshly cooked pasta to gently cook the eggs and form a glossy emulsion with the cheese, creating a silky coating without cream. The dish centers on few ingredients—pasta, cured pork, eggs, and hard cheese—seasoned with black pepper. It’s quick to prepare, satisfying, and endlessly customizable with a good balance of salty, savory, and peppery notes.

Ingredients
| Ingredient | Amount | Notes |
|---|---|---|
| Spaghetti | 12 oz (340 g) | al dente, salted water |
| Eggs | 4 large | traditional is 4 whole eggs (or 2 yolks + 2 whole eggs) |
| Pecorino Romano or Parmesan cheese | 1 cup, finely grated | Pecorino Romano for authenticity if you have it |
| Pancetta or guanciale | 4 oz (115 g), diced | guanciale is most authentic; pancetta also works well |
| Freshly ground black pepper | 1–2 tsp | to taste |
| Salt | to taste | for pasta water; optional beyond salted water |

Instructions
Conventional cooking
| Step | Instruction |
|---|---|
| 1 | Bring a large pot of salted water to a boil. Add spaghetti and cook until al dente according to package directions. Reserve 1/2 cup of the pasta cooking water, then drain the pasta. |
| 2 | In a bowl, whisk together the eggs, grated cheese, and a generous amount of black pepper. |
| 3 | In a large skillet over medium heat, cook the pancetta or guanciale until crisp. Remove from heat. |
| 4 | Return the drained pasta to the pot. Add the crisp pancetta with its rendered fat and toss to coat the noodles. |
| 5 | Off the heat, slowly pour in the egg–cheese mixture, tossing vigorously to create a creamy sauce. Add small amounts of the reserved pasta water as needed to reach a silky emulsion. |
| 6 | Season with additional pepper and, if desired, a little extra cheese. Serve immediately. |

Air fryer cooking
| Step | Instruction |
|---|---|
| 1 | Preheat the air fryer to 400°F (200°C). Place the diced pancetta or guanciale in the air fryer basket and cook 6–8 minutes until crisp. Carefully drain excess fat. |
| 2 | In the meantime, boil a large pot of salted water and cook the spaghetti to al dente. Reserve 1/2 cup pasta water, then drain. |
| 3 | In a bowl, whisk together the eggs, grated cheese, and pepper. |
| 4 | In a large bowl, toss the hot spaghetti with the crisp pancetta and its rendered fat to coat. |
| 5 | Off the heat, slowly add the egg–cheese mixture, tossing vigorously to emulsify with the pasta water until a creamy sauce forms. Add more pasta water a little at a time if needed to loosen the sauce. |
| 6 | Optional: Transfer the sauced pasta to an oven-safe dish and place in the air fryer at 325°F (165°C) for 2–3 minutes to lightly set the sauce, then serve immediately. |

Notes
- The key to success is adding the egg mixture off the heat and using the residual heat of the pasta to gently cook the eggs. Do not hard-crack the eggs; you’re aiming for a silky, creamy sauce.
- If you prefer a firmer bite on the pasta, cook it slightly longer; if you want a looser sauce, add a touch more pasta water.
- For a more traditional flavor, use guanciale instead of pancetta and Pecorino Romano instead of Parmesan.

Enjoy your Spaghetti alla Carbonara, whether you’re cooking on the traditional stovetop or adapting with an air fryer twist.